# ID3、C4.5、CART决策树算法

建议先阅读[决策树概率与基本流程](./summary.ipynb)。

为方便描述和阅读，再贴一遍学习过程。这篇文章就是第8步的详细扩展。

<img src="./decision-tree-algrithom.jpg" width="600">

## 理论

### ID3

信息论与概率统计中，**熵**（entropy）是随机变量不确定性的度量。

设X是一个取值有限的离散随机变量，其概率分布为

$$P(X=x_i) = p_i,\quad i=1,2,\cdots,n$$

则随机变量X的**熵**定义为

$$H(X) = -\sum_{i=1}^{n}p_i \log p_i$$

熵越大，随机变量的不确定性就越大。

设有随机变量 $(X,Y)$，其联合概率分布为

$$P(X=x_i,Y=y_j) = p_{ij},\quad i=1,2,\cdots,n;\ j=1,2\cdots,m$$

**条件熵**（conditional entropy）$H(Y\ |\ X)$ 表示已知随机变量X的条件下随机变量Y的不确定性，其定义为X给定条件下Y的条件概率分布的熵对X的数学期望

$$H(Y\ |\ X) = \sum_{i=1}^{n}p_iH(Y\ |\ X=x_i)$$

这里，$p_i = P(X=x_i),\quad x=1,2,\cdots,n$。

**信息增益**：特征 $a$ 对训练数据集 $D$ 的信息增益 $g(D, a)$，定义为集合 $D$ 的经验熵 $H(D)$ 与特征 $a$ 给定条件下 $D$ 的经验条件熵 $H(D\ |\ a)$ 之差，即

$$g(D, a) = H(D) - H(D\ |\ a)$$

一般地，熵与条件熵之差称为**互信息**（mutual entropy），决策树学习中的信息增益等价于训练数据集中类与特征的互信息。

显然，信息增益 $g(D, a)$ 表示由于特征 $a$ 而使得对数据集 $D$ 的分类的不确定性减少的程度，因此**可以选择信息增益最大的属性作为划分属性**。

设训练数据集为 $D$，$\mid D \mid$ 表示样本个数。有 $K$ 个类 $C_k$，$k=1,2,\cdots,K$，$\mid C_k \mid$ 为属于类 $C_k$ 的样本个数，$ \sum_{i=1}^{K} \mid C_k \mid = \mid D \mid $。设特征 $a$ 有 $n$ 个不同的取值$ \{a^1, a^2, \cdots , a^n\}$，根据特征 $a$ 的取值将 $ D $ 划分为 $ n$ 个子集 $\ D_1, D_2, \cdots, D_n$，$ \mid D_i \mid$ 为 $D_i$ 的样本个数，$\sum_{i=1}^{n} \mid D_i \mid = \mid D \mid $。记子集 $ D_i $ 中属于类 $C_k$ 的样本的集合为 $ D_{ik} $，即 $ D_{ik} = D_i \cap C_k$，$ \mid D_{ik} \mid$ 为 $D_{ik}$ 样本的个数，信息增益的具体计算如下：

-----

**输入**：训练数据集 $D$ 和特征 $a$；
**输出**：特征 $a$ 对训练数据集 $D$ 的信息增益 $g(D, a)$。

1 计算数据集 $D$ 的经验熵

$$H(D) = -\sum_{k=1}^{K} \frac{|C_k|}{|D|}\log_2 \frac{|C_k|}{|D|}$$

2 计算特征 $a$ 对数据集 $D$ 的经验条件熵

$$H(D\ |\ a) = \sum_{i=1}^{n} \frac{|D_i|}{|D|}H(D_i) = -\sum_{i=1}^{n}\frac{|D_i|}{|D|} \sum_{k=1}^{K} \frac{|D_{ik}|}{|D_i|} \log_2 \frac{|D_{ik}|}{|D_i|}$$

3 计算信息增益

$$g(D, a) = H(D) - H(D\ |\ a)$$

------

**ID3决策树学习算法就是以信息增益为准则来选择划分属性**，即在上面伪代码第8行选择属性 $a_* = \mathop{\arg \max}_{a \in A} g(D,a)$。

> ID3中的ID是Iterative Dichotomiser（迭代二分器）的简称。



### C4.5

信息增益准则对可取值数目较多的属性有所偏好，为减少这种偏好可能带来的不利影响，著名的**C4.5**决策树算法不直接使用信息增益，而是使用**增益率**（gain ratio）来选择最优划分属性。

特征a对训练数据集D的增益率 $g_R(D, a)$ 定义为其信息增益 $g(D, a)$ 与训练数据集D关于特征a的值的熵 $H_{a}(D)$ 之比：

$$g_R (D, a) = \frac{g(D, a)}{H_{a}(D)}$$

其中，$H_{a}(D) = - \sum_{i=1}^{n} \frac{\mid D_i \mid}{\mid D \mid} \log_{2} \frac{\mid D_i \mid}{\mid D \mid}$，n是特征a取值的个数。

信息增益比准则对取值数目较少的属性有所偏好。C4.5并不直接选择信息增益比最大的属性进行划分，而是使用了一个启发式：先从候选划分属性中找出信息增益高于平均水平的属性，再从中选择信息增益比最高的。

### 基尼指数与CART

CART（Classification And Regression Tree，分类与回归树）决策树使用**基尼指数**（Gini index）来选择划分属性。

分类问题中，假设有K个类，样本点属于第k类的概率为 $p_k$，则概率分布的基尼指数定义为

$$\begin{align*} \text{Gini}(p) = \sum_{k=1}^{K} p_k (1- p_{k}) = 1 - \sum_{k=1}^{K}p_k^2 \end{align*}$$

对于给定的样本集合D，其基尼指数为

$$\text{Gini} (D) = 1 - \sum_{k=1}^{K} \left( \frac{\mid C_k \mid}{\mid D \mid} \right) ^2$$

这里，$C_k$ 是D中属于第k类的样本子集，K是类的个数。

样本集合D根据属性a的取值可以分为n部分：$D_1, D_2, \ldots, D_n$。则，在属性a的条件下，集合D的基尼指数定义为

$$\text{Gini} (D, a) = \sum_{i=1}^{n} \frac{\mid D_i \mid }{\mid D \mid} \text{Gini} (D_i) $$

基尼指数 $\text{Gini} (D)$ 表示集合D的不确定性，基尼指数 $\text{Gini} (D, a)$  表示经属性a分割后集合D的不确定性。基尼指数越大，样本的不确定性也就越大。


CART决策树在候选属性集合A中，选择那个使得划分后**基尼指数最小**的属性作为最优划分属性，即 $a_* = \mathop{\arg \max}_{a \in A} \text{Gini}(D, a)$。


> **基尼指数与熵**
>
> 首先，对 $-\ln x$在 $x=1$ 处泰勒展开，忽略高阶无穷小，可以得到$-\ln x \approx 1 - x$
>
> 若熵中的对数不是以2为底，而是以e为底：
>
> $$\begin{align*} H(D) &= -\sum_{k=1}^{K}p_k \ln p_k \\ &\approx \sum_{k=1}^{K} p_k (1-p_k) \\ &= \text{Gini}(D) \end{align*} $$

## 示例

## 代码